In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Roma,43.2128,-75.4557,86.95,67,75,10.36,US,2022-07-21 19:10:18
1,1,Ushuaia,-54.8000,-68.3000,31.66,86,40,23.02,AR,2022-07-21 19:07:18
2,2,Tuatapere,-46.1333,167.6833,43.50,90,100,1.28,NZ,2022-07-21 19:11:59
3,3,Rio Grande,-32.0350,-52.0986,66.31,97,100,4.23,BR,2022-07-21 19:11:59
4,4,Khatanga,71.9667,102.5000,49.91,81,100,11.77,RU,2022-07-21 19:06:18


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Roma,43.2128,-75.4557,86.95,67,75,10.36,US,2022-07-21 19:10:18
16,16,Landau,49.2075,8.1133,78.62,52,13,3.71,DE,2022-07-21 19:12:01
17,17,Atuona,-9.8000,-139.0333,77.47,74,18,16.20,PF,2022-07-21 19:07:33
22,22,Kapaa,22.0752,-159.3190,80.19,74,75,18.41,US,2022-07-21 19:07:36
23,23,Aden,12.7794,45.0367,85.23,62,9,3.11,YE,2022-07-21 19:12:03
25,25,Hilo,19.7297,-155.0900,79.29,99,75,10.36,US,2022-07-21 19:06:25
26,26,Myanaung,18.2833,95.3167,79.20,87,100,3.85,MM,2022-07-21 19:12:04
31,31,Kilindoni,-7.9139,39.6668,75.88,72,0,16.53,TZ,2022-07-21 19:12:05
32,32,Bafoulabe,13.8065,-10.8321,80.33,81,100,2.95,ML,2022-07-21 19:07:24
34,34,Pafos,34.7667,32.4167,82.18,69,0,6.91,CY,2022-07-21 19:10:54


In [11]:
preferred_cities_df.count()

City_ID       205
City          205
Lat           205
Lng           205
Max Temp      205
Humidity      205
Cloudiness    205
Wind Speed    205
Country       205
Date          205
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Roma,US,86.95,43.2128,-75.4557,
16,Landau,DE,78.62,49.2075,8.1133,
17,Atuona,PF,77.47,-9.8000,-139.0333,
22,Kapaa,US,80.19,22.0752,-159.3190,
23,Aden,YE,85.23,12.7794,45.0367,
25,Hilo,US,79.29,19.7297,-155.0900,
26,Myanaung,MM,79.20,18.2833,95.3167,
31,Kilindoni,TZ,75.88,-7.9139,39.6668,
32,Bafoulabe,ML,80.33,13.8065,-10.8321,
34,Pafos,CY,82.18,34.7667,32.4167,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

IndexError: list index out of range

In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))